# Comment Category

- In this notebook, I prepared a batch to classify each thesis-relevant comment into one or more thematic categories.  
- I used the rewritten comment text to build the batch JSONL file.  
- The prompt asked GPT-4o to assign one or more relevant category labels (e.g., Sportswashing, Human Rights, Financial Ethics, Corruption, etc.).  
- Due to the token limit, I split the batch into two parts before submitting it to OpenAI’s Batch API.  
- Once the outputs were ready, I downloaded the JSONL files and extracted the `custom_id` and assigned categories.  
- I saved the results to a CSV for further analysis, allowing me to see which themes were most commonly associated with the comments.


### Importing Libraries

In [ ]:
import pandas as pd
import json
from tqdm import tqdm
import os
import openai

In [1]:
# Loading the full dataset
df = pd.read_csv("Thesis_Relevant_With_Transcript_Influence_and_Agreement.csv")
print(f"Loaded {len(df)} comments.")

Loaded 71441 comments.


/var/folders/nv/wkg5hpgj1kl1012mfw_qwby80000gn/T/ipykernel_13070/3311419059.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Thesis_Relevant_With_Transcript_Influence_and_Agreement.csv")


### Creating the Json file

In [5]:
output_jsonl = "batch_step_category.jsonl"

# Opening the file and writing each row as a separate task
with open(output_jsonl, "w", encoding="utf-8") as f:
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Building Category batch"):
        comment_id = str(row["Comment_ID"])
        comment = row["Rewritten Comment"]

        # Building the request for GPT-4o Mini
        task = {
            "custom_id": comment_id,  
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o-mini",
                "messages": [
                    {
                        "role": "system",
                        "content": "You are helping a researcher label YouTube comments based on content themes."
                    },
                    {
                        "role": "user",
                        "content": f"""Comment: "{comment}"\n\nWhat category does this comment fall under?\n\nReply ONLY with one label:\n- Sportswashing\n- Human Rights\n- Financial Ethics\n- Corruption\n- Geopolitics\n- Media Criticism\n- Other"""
                    }
                ],
                "temperature": 0,        
                "max_tokens": 10         
            }
        }


        f.write(json.dumps(task) + "\n")

print(f" Saved: {output_jsonl}")


Building Category batch: 100%|████████████████████████| 71441/71441 [00:07<00:00, 9702.39it/s]
Saved: batch_step_category.jsonl


### Splitting the Batch due to Token Limit

In [9]:
input_file = "batch_step_category.jsonl"  # Change to your file
output_file_1 = input_file.replace(".jsonl", "_part1.jsonl")
output_file_2 = input_file.replace(".jsonl", "_part2.jsonl")

# Reading the file
with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Splitting
mid = len(lines) // 2
part1 = lines[:mid]
part2 = lines[mid:]

# Writing to two new files
with open(output_file_1, "w", encoding="utf-8") as f:
    f.writelines(part1)

with open(output_file_2, "w", encoding="utf-8") as f:
    f.writelines(part2)

print(f"Saved {len(part1)} → {output_file_1}")
print(f"Saved {len(part2)} → {output_file_2}")


Saved 35720 → batch_step_category_part1.jsonl
Saved 35721 → batch_step_category_part2.jsonl


### Submitting Batches to Openai

In [11]:
openai.api_key = "***********"
client = openai.OpenAI(api_key=openai.api_key)

batch_files = [
    "batch_step_category_part1.jsonl",
    "batch_step_category_part2.jsonl"
]

# Submitting batches
for i, file_path in enumerate(batch_files):
    try:
        print(f"\nSubmitting batch {i+1}/{len(batch_files)}: {file_path}")

        # Uploading file
        with open(file_path, "rb") as f:
            upload = client.files.create(file=f, purpose="batch")
        print(f" Uploaded: File ID = {upload.id}")

        # Creating batch job
        batch = client.batches.create(
            input_file_id=upload.id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
        )
        print(f"Batch submitted: Batch ID = {batch.id}")

    except Exception as e:
        print(f"Failed to submit {file_path}: {e}")


Submitting batch 1/2: batch_step_category_part1.jsonl
Uploaded: File ID = file-1tDvgdUQiVD8C8huVsQMKK
Batch submitted: Batch ID = batch_67e5d1a9f8188190b0964f2a67ee6dce
Submitting batch 2/2: batch_step_category_part2.jsonl
Uploaded: File ID = file-SfF2bUr2XwLNLdjPWie1rg
Batch submitted: Batch ID = batch_67e5d1b3508081908414d40cb6ee3e24


### Downloading Json Output from Openai

In [20]:
openai.api_key = "*****************" 

# List of output file IDs from completed batches
output_file_ids = [
    "file-8xjkdD9kzDg42E8TonAQj5",
    "file-Sk7PUnXQRgqtotqVpVsHX3"
]

# Downloading each output file properly
for file_id in output_file_ids:
    file_response = openai.files.content(file_id)

    # Saving the file locally in binary mode
    output_filename = f"{file_id}.jsonl"
    with open(output_filename, "wb") as f:
        for chunk in file_response.iter_bytes():
            f.write(chunk)
    
    print(f"File downloaded: {output_filename}")

File downloaded: file-8xjkdD9kzDg42E8TonAQj5.jsonl
File downloaded: file-Sk7PUnXQRgqtotqVpVsHX3.jsonl


### Converting Json Output to a CSV file

In [23]:
jsonl_files = [
    "file-8xjkdD9kzDg42E8TonAQj5.jsonl",
    "file-Sk7PUnXQRgqtotqVpVsHX3.jsonl"
]

# Storing extracted data
extracted_data = []

# Looping through files
for jsonl_file in jsonl_files:
    if not os.path.exists(jsonl_file):
        print(f"{jsonl_file} not found")
        continue

    print(f"Processing {jsonl_file}")

    with open(jsonl_file, "r", encoding="utf-8") as f:
        for line in f:
            json_obj = json.loads(line)

            custom_id = json_obj.get("custom_id", "")
            response = json_obj.get("response", {}).get("body", {}).get("choices", [{}])[0].get("message", {}).get("content", "").strip()

            extracted_data.append({
                "custom_id": custom_id,
                "Category": response
            })

# Creating and saving CSV
df = pd.DataFrame(extracted_data)
output_csv = "category_responses.csv"
df.to_csv(output_csv, index=False)

print(f"\nExtracted responses saved to: {output_csv}")
print("Category breakdown:")
print(df["Category"].value_counts())


Processing file-8xjkdD9kzDg42E8TonAQj5.jsonl
Processing file-Sk7PUnXQRgqtotqVpVsHX3.jsonl

Extracted responses saved to: category_responses.csv
Category breakdown:
Category
Geopolitics               17605
Human Rights              16978
Other                     15450
Financial Ethics           8598
Media Criticism            5665
Corruption                 5016
Sportswashing              2128
Environmental Concerns        1
Name: count, dtype: int64
